<a href="https://githubtocolab.com/giswqs/gee-tutorials/blob/master/Analysis/zonal_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

In [1]:
import ee
import geemap
import os

## Create an interactive map

In [5]:
Map = geemap.Map()
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-JiQhWls7XZvyLTKN77MNlCZ2BgJBarojq-hKi5B5tg&tc=PwnWOu8oBCF7quxDfT46PjRpaME_kl-NppMKMWWq1PU&cc=ZqG_vZrQQ2dPr7aD9ZI_bE50SYUgJf0ml7Ceb-1NjbI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXk7zETGAvJSvgvjEvLPBK9-7bEJp6C8mktSJML2vU8vs6V6pmp6ESs

Successfully saved authorization token.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Add Earth Engine data

In [15]:
# Vis Parameter for NDVI
ndviViz   = { 'min' : 0,
                  'max' : 1,
                  'palette' : ['red','green']
                }
# Feature Collection
plot36    = ee.FeatureCollection("projects/ee-krnwnaji1995/assets/Assets_GEE_EPEL_WKS")
Map.addLayer(plot36,{},'plot')
Map.centerObject(plot36)

In [10]:
imageColl = (ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia")
                .filterDate("2021-09-01","2023-09-30")
                .filterBounds(plot36))


                # .map(function(image){
                #   var ndvi = image.normalizedDifference(['N','R']).rename('ndvi')
                #   return image.addBands(ndvi)
                # })
                # .map(function(a){
                #   return a
                #       .set('month',ee.Image(a).date().get('month'))
                #       .set('year',ee.Image(a).date().get('year'))
                #       .set('imageId',ee.String('NICFI_').cat(ee.Image(a).date().get('year')).cat('_').cat(ee.Image(a).date().get('month')))
                # })

In [ ]:
# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
dem_vis = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Engine DEM to map
Map.addLayer(dem, dem_vis, 'SRTM DEM')

# Add Landsat data to map
landsat = ee.Image('LE7_TOA_5YEAR/1999_2003')

landsat_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'gamma': 1.4
}
Map.addLayer(landsat, landsat_vis, "Landsat", False)

states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, 'US States')

## Compute zonal statistics for one image

In [ ]:
out_dir = os.path.expanduser('~/Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
out_dem_stats = os.path.join(out_dir, 'dem_stats.csv')

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(dem, states, out_dem_stats, statistics_type='MEAN', scale=1000)

In [ ]:
out_landsat_stats = os.path.join(out_dir, 'landsat_stats.csv')
geemap.zonal_statistics(landsat, states, out_landsat_stats, statistics_type='SUM', scale=1000)

## Compute zonal statistics for time-series images

In [ ]:
Map = geemap.Map()

collection = ee.ImageCollection('MODIS/MCD43A4_006_NDVI') \
                  .filter(ee.Filter.date('2018-04-01', '2018-05-01')) \
                  .select("NDVI")\

vis_params = {
  'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

first_image = collection.first()

Map.addLayer(first_image, vis_params, "First image")
Map.setCenter(-7.03125, 31.0529339857, 2)
Map

In [ ]:
modis = collection.toBands()
Map.addLayer(modis, {}, "MODIS Time series", False)

In [ ]:
out_landsat_stats = os.path.join(out_dir, 'ndvi.csv')
geemap.zonal_statistics(modis, states, out_landsat_stats, statistics_type='MEAN', scale=1000)